<a href="https://colab.research.google.com/github/fjadidi2001/fake_news_detection/blob/main/DansE_Mar29.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data

# Mount Drive in Colab

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Load the Dataset



In [2]:
import pandas as pd
import numpy as np

# Replace with your actual file path
file_path = '/content/drive/MyDrive/Projects/Hayat/facebook-fact-check.csv'


df = pd.read_csv(file_path, encoding='latin-1')

# Initial Data Inspection

In [3]:
print(df.head(2))  # See first 2 rows
print("\nMissing values:\n", df.isnull().sum())

     account_id       post_id    Category               Page  \
0  1.840000e+14  1.040000e+15  mainstream  ABC News Politics   
1  1.840000e+14  1.040000e+15  mainstream  ABC News Politics   

                                            Post URL Date Published Post Type  \
0  https://www.facebook.com/ABCNewsPolitics/posts...      9/19/2016     video   
1  https://www.facebook.com/ABCNewsPolitics/posts...      9/19/2016      link   

               Rating Debate  share_count  reaction_count  comment_count  \
0  no factual content    NaN          NaN           146.0           15.0   
1         mostly true    NaN          1.0            33.0           34.0   

                                        Context Post  
0  WATCH: &quot;JEB EXCLAMATION POINT!&quot; - Je...  
1  Can either candidate move the needle in the de...  

Missing values:
 account_id           0
post_id              0
Category             0
Page                 0
Post URL             0
Date Published       0
Post Type    

# Handle Missing Values

In [4]:
# Strategy 1: Fill categorical columns
df['Rating'] = df['Rating'].fillna('Unknown')
df['Debate'] = df['Debate'].fillna('Not Specified')

# Strategy 2: Fill numerical columns with median
numeric_cols = ['share_count', 'reaction_count', 'comment_count']
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].median())

# Alternative: Drop rows with critical missing values
# df = df.dropna(subset=['important_column'])

# Additional Preprocessing

In [7]:
# Convert date to datetime format
df['Date Published'] = pd.to_datetime(df['Date Published'], format='%m/%d/%Y')

# Clean text columns
df['Context Post'] = df['Context Post'].str.replace('"', '')

In [8]:
df['account_id'] = df['account_id'].astype(str)
df['post_id'] = df['post_id'].astype(str)

In [9]:
categorical_cols = ['Category', 'Page', 'Post Type']
df[categorical_cols] = df[categorical_cols].fillna('Unknown')

In [10]:
print(df.info())
print(df.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2282 entries, 0 to 2281
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   account_id      2282 non-null   object        
 1   post_id         2282 non-null   object        
 2   Category        2282 non-null   object        
 3   Page            2282 non-null   object        
 4   Post URL        2282 non-null   object        
 5   Date Published  2282 non-null   datetime64[ns]
 6   Post Type       2282 non-null   object        
 7   Rating          2282 non-null   object        
 8   Debate          2282 non-null   object        
 9   share_count     2282 non-null   float64       
 10  reaction_count  2282 non-null   float64       
 11  comment_count   2282 non-null   float64       
 12  Context Post    2282 non-null   object        
dtypes: datetime64[ns](1), float64(3), object(9)
memory usage: 231.9+ KB
None
account_id        0
post_id        

In [11]:
def processElement(elem):
    id_line = elem[0]
    text = elem[1]
    # Use 'Context Post' instead of 'content' if needed
    text = tkn.createCorpus(text, remove_stopwords=False)
    return id_line, text

# Main source

In [17]:
from google.colab import drive
drive.mount('/content/drive')

# Download and save to Drive (run once)
# !wget http://nlp.stanford.edu/data/glove.6B.zip -O /content/drive/MyDrive/glove.6B.zip
!unzip /content/drive/MyDrive/glove.6B.zip -d /content/drive/MyDrive/glove

# Load from Drive in future sessions
embeddings_index = {}
with open('/content/drive/MyDrive/glove/glove.6B.100d.txt', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

print(f'Found {len(embeddings_index)} word vectors.')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Archive:  /content/drive/MyDrive/glove.6B.zip
  inflating: /content/drive/MyDrive/glove/glove.6B.50d.txt  
  inflating: /content/drive/MyDrive/glove/glove.6B.100d.txt  
  inflating: /content/drive/MyDrive/glove/glove.6B.200d.txt  
  inflating: /content/drive/MyDrive/glove/glove.6B.300d.txt  
Found 400000 word vectors.


## wordembeddings

In [ ]:
# Step 1: Clean up the environment
!pip uninstall -y numpy mittens gensim scipy smart-open wrapt tensorflow tensorflow-datasets dm-tree numba

# Step 2: Install compatible versions
!pip install numpy==1.26.4 mittens==0.2 gensim==4.3.3 scipy==1.13.1 smart-open==7.1.0 wrapt==1.17.2

# Step 3: Restart runtime (run this once, then comment out)
import os
os.kill(os.getpid(), 9)

# Step 4: After restart, run the code
import numpy as np
print("NumPy version:", np.__version__)

from gensim.models import Word2Vec, FastText
import gensim
print("Gensim version:", gensim.__version__)

from mittens import Mittens, GloVe
import mittens
print("Mittens version:", mittens.__version__)

Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2
Found existing installation: scipy 1.14.1
Uninstalling scipy-1.14.1:
  Successfully uninstalled scipy-1.14.1
Found existing installation: smart-open 7.1.0
Uninstalling smart-open-7.1.0:
  Successfully uninstalled smart-open-7.1.0
Found existing installation: wrapt 1.17.2
Uninstalling wrapt-1.17.2:
  Successfully uninstalled wrapt-1.17.2
Found existing installation: tensorflow 2.18.0
Uninstalling tensorflow-2.18.0:
  Successfully uninstalled tensorflow-2.18.0
Found existing installation: tensorflow-datasets 4.9.8
Uninstalling tensorflow-datasets-4.9.8:
  Successfully uninstalled tensorflow-datasets-4.9.8
Found existing installation: dm-tree 0.1.9
Uninstalling dm-tree-0.1.9:
  Successfully uninstalled dm-tree-0.1.9
Found existing installation: numba 0.60.0
Uninstalling numba-0.60.0:
  Successfully uninstalled numba-0.60.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.0

In [2]:
# Verify versions after restart
import numpy as np
print("NumPy version:", np.__version__)

from gensim.models import Word2Vec, FastText
import gensim
print("Gensim version:", gensim.__version__)

from mittens import Mittens, GloVe
import mittens
print("Mittens version:", mittens.__version__)
import osfit(cooccurrence, vocab=None, initial_embedding_dict=None)
class WordEmbeddings:

    def __init__(self, corpus, normalize_tfidf=False):
        self.corpus = corpus
        self.normalize_tfidf = normalize_tfidf
        self.documents = []
        self.sentences = []
        self.word2id = {}
        self.no_words = 0
        self.max_size = 0
        self.no_docs = len(self.corpus)

    def prepareDocuments(self):
        word_id = 1
        for document in self.corpus:
            doc = []
            for sentence in document:
                self.sentences.append(sentence)
                for word in sentence:
                    if self.word2id.get(word) is None:
                        self.word2id[word] = word_id
                        word_id += 1
                    doc.append(self.word2id[word])
            if self.max_size < len(doc):
                self.max_size = len(doc)
            self.documents.append(doc)

        self.no_words = len(self.word2id) + 1
        return self.documents

    def word2vecEmbedding(self, window_size=10, no_components=128, epochs=10, workers=os.cpu_count(), sg=0, learning_rate=0.05):
        self.word2vec = np.empty(shape=(self.no_words, no_components))
        model = Word2Vec(self.sentences, vector_size=no_components, window=window_size, min_count=1,
                         workers=workers, sg=sg, alpha=learning_rate, epochs=epochs)
        self.word2vec[0] = np.zeros(no_components)
        for word in self.word2id:
            self.word2vec[self.word2id[word]] = model.wv[word]
        return self.word2vec

    def word2GloVeEmbedding(self, window_size=10, no_components=128, epochs=10, workers=os.cpu_count(), learning_rate=0.05):
        self.word2glove = np.empty(shape=(self.no_words, no_components))
        model = GloVe(n=no_components, learning_rate=learning_rate)

        vocab = list(self.word2id.keys())
        cooc_matrix = np.zeros((len(vocab), len(vocab)))
        for sentence in self.sentences:
            for i, word in enumerate(sentence):
                word_idx = self.word2id[word] - 1
                for j in range(max(0, i - window_size), min(len(sentence), i + window_size + 1)):
                    if i != j:
                        cooc_idx = self.word2id[sentence[j]] - 1
                        cooc_matrix[word_idx, cooc_idx] += 1

        embeddings = model.fit(cooc_matrix)
        self.word2glove[0] = np.zeros(no_components)
        for word, idx in self.word2id.items():
            self.word2glove[idx] = embeddings[idx - 1]
        return self.word2glove

    def word2FastTextEmbeddings(self, window_size=10, no_components=128, epochs=10, workers=os.cpu_count(), sg=0, learning_rate=0.05):
        self.word2fasttext = np.empty(shape=(self.no_words, no_components))
        model = FastText(self.sentences, vector_size=no_components, window=window_size, min_count=1,
                         workers=workers, sg=sg, alpha=learning_rate, epochs=epochs)
        self.word2fasttext[0] = np.zeros(no_components)
        for word in self.word2id:
            self.word2fasttext[self.word2id[word]] = model.wv[word]
        return self.word2fasttext

    def word2MittensEmbedding(self, window_size=10, no_components=128, epochs=10, workers=os.cpu_count(), learning_rate=0.05):
        self.word2mittens = np.empty(shape=(self.no_words, no_components))
        vocab = list(self.word2id.keys())
        cooc_matrix = np.zeros((len(vocab), len(vocab)))
        for sentence in self.sentences:
            for i, word in enumerate(sentence):
                word_idx = self.word2id[word] - 1
                for j in range(max(0, i - window_size), min(len(sentence), i + window_size + 1)):
                    if i != j:
                        cooc_idx = self.word2id[sentence[j]] - 1
                        cooc_matrix[word_idx, cooc_idx] += 1

        model = Mittens(n=no_components, max_iter=epochs, mittens=learning_rate)
        embeddings = model.fit(cooc_matrix, vocab=vocab)
        self.word2mittens[0] = np.zeros(no_components)
        for word, idx in self.word2id.items():
            self.word2mittens[idx] = embeddings[idx - 1]
        return self.word2mittens

if __name__ == '__main__':
    corpus = [
        [['Hello', 'this', 'tutorial', 'on', 'how', 'convert', 'word', 'integer', 'format'],
         ['this', 'beautiful', 'day'],
         ['Jack', 'going', 'office']],
        [['Hello', 'this', 'tutorial', 'on', 'how', 'convert', 'word', 'integer', 'format'],
         ['this', 'beautiful', 'day'],
         ['Jack', 'going', 'office']],
        [['Hello', 'this', 'tutorial', 'on', 'how', 'convert', 'word', 'integer', 'format'],
         ['this', 'beautiful', 'day'],
         ['Jack', 'going', 'office']],
    ]

    we = WordEmbeddings(corpus)
    docs = we.prepareDocuments()
    print(np.array(docs, dtype=object).shape)
    print(docs)

    w2v = we.word2vecEmbedding()
    print("Word2Vec:", w2v.shape)
    print(w2v)

    w2f = we.word2FastTextEmbeddings()
    print("FastText:", w2f.shape)
    print(w2f)

    w2g = we.word2GloVeEmbedding()
    print("GloVe:", w2g.shape)
    print(w2g)

    w2m = we.word2MittensEmbedding()
    print("Mittens:", w2m.shape)
    print(w2m)

    print("\n\nComparison for word ID 1:")
    print("Word2Vec:", w2v[1])
    print("FastText:", w2f[1])
    print("GloVe:", w2g[1])
    print("Mittens:", w2m[1])

NumPy version: 1.26.4
Gensim version: 4.3.3
Mittens version: 0.2
(3, 15)
[[1, 2, 3, 4, 5, 6, 7, 8, 9, 2, 10, 11, 12, 13, 14], [1, 2, 3, 4, 5, 6, 7, 8, 9, 2, 10, 11, 12, 13, 14], [1, 2, 3, 4, 5, 6, 7, 8, 9, 2, 10, 11, 12, 13, 14]]
Word2Vec: (15, 128)
[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [-6.21626532e-05  3.19088437e-03 -7.12745590e-03 ...  9.92520479e-04
   7.14840600e-03  2.83964048e-03]
 [-3.82514030e-04  1.51936125e-04  3.97204095e-03 ...  2.31946167e-03
  -3.84517061e-03  3.48054501e-03]
 ...
 [-1.44330040e-03 -3.37802432e-03 -5.04739862e-03 ...  4.78180777e-03
   3.40414606e-03  2.03621481e-03]
 [ 5.68562094e-03 -4.50939965e-03  6.46589976e-03 ...  4.88198549e-03
  -7.37946481e-03  7.49228429e-03]
 [-1.35904271e-03  5.24326880e-03  7.78503902e-03 ... -6.09789602e-03
  -7.12639559e-03 -4.63685114e-03]]


Iteration 100: error 0.0000

FastText: (15, 128)
[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [-1.05216913e-03 -7.23858946e-04 -2.48263852e-04 ...  4.79545008e-04
   1.65466184e-03 -2.04305412e-04]
 [-1.36351539e-03  1.12285069e-03 -1.62398699e-03 ...  1.27794396e-03
  -4.78518370e-04  1.83548301e-03]
 ...
 [-2.22318270e-03  2.95931986e-05 -4.96662979e-04 ...  1.28600537e-03
   9.15645447e-04 -2.91286968e-04]
 [-3.23804357e-04  9.54689051e-04 -1.18813978e-03 ...  6.51234936e-04
   2.29042576e-04  1.05157425e-03]
 [-4.99513757e-04  2.49632495e-03  2.18920293e-03 ...  4.68000828e-04
  -1.34677067e-03 -1.70546729e-04]]
GloVe: (15, 128)
[[ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.35759968 -0.41002173 -0.11296739 ... -0.13695379 -0.08978107
  -0.02064198]
 [ 0.03458208 -0.26300714  0.31167143 ... -0.42121385  0.17037075
   0.06615572]
 ...
 [ 0.00209569  0.26936345  0.07195031 ...  0.01113254  0.02607798
   0.16050002]
 [ 0.0

AttributeError: 'Mittens' object has no attribute 'has_embedding'

In [5]:
# Verify versions after restart
import numpy as np
print("NumPy version:", np.__version__)

from gensim.models import Word2Vec, FastText
import gensim
print("Gensim version:", gensim.__version__)

from mittens import Mittens, GloVe
import mittens
print("Mittens version:", mittens.__version__)

# Minimal test for Mittens
vocab = ['hello', 'world']
cooc_matrix = np.array([[0, 1], [1, 0]])  # Simple co-occurrence matrix
mittens_model = Mittens(n=10, max_iter=10)
try:
    embeddings = mittens_model.fit(cooc_matrix, vocab=vocab)
    print("Mittens test successful:", embeddings.shape)
except Exception as e:
    print("Mittens test failed:", str(e))

class WordEmbeddings:

    def __init__(self, corpus, normalize_tfidf=False):
        self.corpus = corpus
        self.normalize_tfidf = normalize_tfidf
        self.documents = []
        self.sentences = []
        self.word2id = {}
        self.no_words = 0
        self.max_size = 0
        self.no_docs = len(self.corpus)

    def prepareDocuments(self):
        word_id = 1
        for document in self.corpus:
            doc = []
            for sentence in document:
                self.sentences.append(sentence)
                for word in sentence:
                    if self.word2id.get(word) is None:
                        self.word2id[word] = word_id
                        word_id += 1
                    doc.append(self.word2id[word])
            if self.max_size < len(doc):
                self.max_size = len(doc)
            self.documents.append(doc)

        self.no_words = len(self.word2id) + 1
        return self.documents

    def word2vecEmbedding(self, window_size=10, no_components=128, epochs=10, workers=os.cpu_count(), sg=0, learning_rate=0.05):
        self.word2vec = np.empty(shape=(self.no_words, no_components))
        model = Word2Vec(self.sentences, vector_size=no_components, window=window_size, min_count=1,
                         workers=workers, sg=sg, alpha=learning_rate, epochs=epochs)
        self.word2vec[0] = np.zeros(no_components)
        for word in self.word2id:
            self.word2vec[self.word2id[word]] = model.wv[word]
        return self.word2vec

    def word2GloVeEmbedding(self, window_size=10, no_components=128, epochs=10, workers=os.cpu_count(), learning_rate=0.05):
        self.word2glove = np.empty(shape=(self.no_words, no_components))
        model = GloVe(n=no_components, learning_rate=learning_rate)

        vocab = list(self.word2id.keys())
        cooc_matrix = np.zeros((len(vocab), len(vocab)))
        for sentence in self.sentences:
            for i, word in enumerate(sentence):
                word_idx = self.word2id[word] - 1
                for j in range(max(0, i - window_size), min(len(sentence), i + window_size + 1)):
                    if i != j:
                        cooc_idx = self.word2id[sentence[j]] - 1
                        cooc_matrix[word_idx, cooc_idx] += 1

        embeddings = model.fit(cooc_matrix)
        self.word2glove[0] = np.zeros(no_components)
        for word, idx in self.word2id.items():
            self.word2glove[idx] = embeddings[idx - 1]
        return self.word2glove

    def word2FastTextEmbeddings(self, window_size=10, no_components=128, epochs=10, workers=os.cpu_count(), sg=0, learning_rate=0.05):
        self.word2fasttext = np.empty(shape=(self.no_words, no_components))
        model = FastText(self.sentences, vector_size=no_components, window=window_size, min_count=1,
                         workers=workers, sg=sg, alpha=learning_rate, epochs=epochs)
        self.word2fasttext[0] = np.zeros(no_components)
        for word in self.word2id:
            self.word2fasttext[self.word2id[word]] = model.wv[word]
        return self.word2fasttext

    def word2MittensEmbedding(self, window_size=10, no_components=128, epochs=10, workers=os.cpu_count(), learning_rate=0.05):
        self.word2mittens = np.empty(shape=(self.no_words, no_components))
        vocab = list(self.word2id.keys())
        cooc_matrix = np.zeros((len(vocab), len(vocab)))
        for sentence in self.sentences:
            for i, word in enumerate(sentence):
                word_idx = self.word2id[word] - 1
                for j in range(max(0, i - window_size), min(len(sentence), i + window_size + 1)):
                    if i != j:
                        cooc_idx = self.word2id[sentence[j]] - 1
                        cooc_matrix[word_idx, cooc_idx] += 1

        model = Mittens(n=no_components, max_iter=epochs, mittens=learning_rate)
        try:
            embeddings = model.fit(cooc_matrix, vocab=vocab)  # Explicitly test this
            self.word2mittens[0] = np.zeros(no_components)
            for word, idx in self.word2id.items():
                self.word2mittens[idx] = embeddings[idx - 1]
        except AttributeError as e:
            print(f"Error in Mittens.fit: {e}")
            embeddings = np.zeros((len(vocab), no_components))  # Fallback
            self.word2mittens[0] = np.zeros(no_components)
            for i, word in enumerate(vocab, 1):
                self.word2mittens[i] = embeddings[i - 1]
        return self.word2mittens

if __name__ == '__main__':
    corpus = [
        [['Hello', 'this', 'tutorial', 'on', 'how', 'convert', 'word', 'integer', 'format'],
         ['this', 'beautiful', 'day'],
         ['Jack', 'going', 'office']],
        [['Hello', 'this', 'tutorial', 'on', 'how', 'convert', 'word', 'integer', 'format'],
         ['this', 'beautiful', 'day'],
         ['Jack', 'going', 'office']],
        [['Hello', 'this', 'tutorial', 'on', 'how', 'convert', 'word', 'integer', 'format'],
         ['this', 'beautiful', 'day'],
         ['Jack', 'going', 'office']],
    ]

    we = WordEmbeddings(corpus)
    docs = we.prepareDocuments()
    print(np.array(docs, dtype=object).shape)
    print(docs)

    w2v = we.word2vecEmbedding()
    print("Word2Vec:", w2v.shape)
    print(w2v)

    w2f = we.word2FastTextEmbeddings()
    print("FastText:", w2f.shape)
    print(w2f)

    w2g = we.word2GloVeEmbedding()
    print("GloVe:", w2g.shape)
    print(w2g)

    w2m = we.word2MittensEmbedding()
    print("Mittens:", w2m.shape)
    print(w2m)

    print("\n\nComparison for word ID 1:")
    print("Word2Vec:", w2v[1])
    print("FastText:", w2f[1])
    print("GloVe:", w2g[1])
    print("Mittens:", w2m[1])

NumPy version: 1.26.4
Gensim version: 4.3.3
Mittens version: 0.2
Mittens test failed: 'Mittens' object has no attribute 'has_embedding'
(3, 15)
[[1, 2, 3, 4, 5, 6, 7, 8, 9, 2, 10, 11, 12, 13, 14], [1, 2, 3, 4, 5, 6, 7, 8, 9, 2, 10, 11, 12, 13, 14], [1, 2, 3, 4, 5, 6, 7, 8, 9, 2, 10, 11, 12, 13, 14]]
Word2Vec: (15, 128)
[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [-6.21626532e-05  3.19088437e-03 -7.12745590e-03 ...  9.92520479e-04
   7.14840600e-03  2.83964048e-03]
 [-3.82514030e-04  1.51936125e-04  3.97204095e-03 ...  2.31946167e-03
  -3.84517061e-03  3.48054501e-03]
 ...
 [-1.44330040e-03 -3.37802432e-03 -5.04739862e-03 ...  4.78180777e-03
   3.40414606e-03  2.03621481e-03]
 [ 5.68562094e-03 -4.50939965e-03  6.46589976e-03 ...  4.88198549e-03
  -7.37946481e-03  7.49228429e-03]
 [-1.35904271e-03  5.24326880e-03  7.78503902e-03 ... -6.09789602e-03
  -7.12639559e-03 -4.63685114e-03]]


Iteration 100: error 0.0000

FastText: (15, 128)
[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [-1.05216913e-03 -7.23858946e-04 -2.48263852e-04 ...  4.79545008e-04
   1.65466184e-03 -2.04305412e-04]
 [-1.36351539e-03  1.12285069e-03 -1.62398699e-03 ...  1.27794396e-03
  -4.78518370e-04  1.83548301e-03]
 ...
 [-2.22318270e-03  2.95931986e-05 -4.96662979e-04 ...  1.28600537e-03
   9.15645447e-04 -2.91286968e-04]
 [-3.23804357e-04  9.54689051e-04 -1.18813978e-03 ...  6.51234936e-04
   2.29042576e-04  1.05157425e-03]
 [-4.99513757e-04  2.49632495e-03  2.18920293e-03 ...  4.68000828e-04
  -1.34677067e-03 -1.70546729e-04]]
GloVe: (15, 128)
[[ 0.          0.          0.         ...  0.          0.
   0.        ]
 [-0.24156722  0.19728762  0.28115077 ...  0.25290197 -0.28166244
   0.03103951]
 [-0.20704135 -0.10191444  0.2596386  ...  0.16890988  0.03980409
   0.13393758]
 ...
 [-0.10637504 -0.02374604 -0.0016289  ...  0.03591276  0.28356568
   0.06366436]
 [-0.0

In [7]:
# Verify versions after restart
import numpy as np
print("NumPy version:", np.__version__)

from gensim.models import Word2Vec, FastText
import gensim
print("Gensim version:", gensim.__version__)

from mittens import Mittens, GloVe
import mittens
print("Mittens version:", mittens.__version__)

from scipy.sparse import csr_matrix  # Add sparse matrix support

class WordEmbeddings:

    def __init__(self, corpus, normalize_tfidf=False):
        self.corpus = corpus
        self.normalize_tfidf = normalize_tfidf
        self.documents = []
        self.sentences = []
        self.word2id = {}
        self.no_words = 0
        self.max_size = 0
        self.no_docs = len(self.corpus)

    def prepareDocuments(self):
        word_id = 1
        for document in self.corpus:
            doc = []
            for sentence in document:
                self.sentences.append(sentence)
                for word in sentence:
                    if self.word2id.get(word) is None:
                        self.word2id[word] = word_id
                        word_id += 1
                    doc.append(self.word2id[word])
            if self.max_size < len(doc):
                self.max_size = len(doc)
            self.documents.append(doc)

        self.no_words = len(self.word2id) + 1
        return self.documents

    def word2vecEmbedding(self, window_size=10, no_components=128, epochs=10, workers=os.cpu_count(), sg=0, learning_rate=0.05):
        self.word2vec = np.empty(shape=(self.no_words, no_components))
        model = Word2Vec(self.sentences, vector_size=no_components, window=window_size, min_count=1,
                         workers=workers, sg=sg, alpha=learning_rate, epochs=epochs)
        self.word2vec[0] = np.zeros(no_components)
        for word in self.word2id:
            self.word2vec[self.word2id[word]] = model.wv[word]
        return self.word2vec

    def word2GloVeEmbedding(self, window_size=10, no_components=128, epochs=10, workers=os.cpu_count(), learning_rate=0.05):
        self.word2glove = np.empty(shape=(self.no_words, no_components))
        model = GloVe(n=no_components, learning_rate=learning_rate)

        vocab = list(self.word2id.keys())
        cooc_matrix = np.zeros((len(vocab), len(vocab)))
        for sentence in self.sentences:
            for i, word in enumerate(sentence):
                word_idx = self.word2id[word] - 1
                for j in range(max(0, i - window_size), min(len(sentence), i + window_size + 1)):
                    if i != j:
                        cooc_idx = self.word2id[sentence[j]] - 1
                        cooc_matrix[word_idx, cooc_idx] += 1

        embeddings = model.fit(cooc_matrix)
        self.word2glove[0] = np.zeros(no_components)
        for word, idx in self.word2id.items():
            self.word2glove[idx] = embeddings[idx - 1]
        return self.word2glove

    def word2FastTextEmbeddings(self, window_size=10, no_components=128, epochs=10, workers=os.cpu_count(), sg=0, learning_rate=0.05):
        self.word2fasttext = np.empty(shape=(self.no_words, no_components))
        model = FastText(self.sentences, vector_size=no_components, window=window_size, min_count=1,
                         workers=workers, sg=sg, alpha=learning_rate, epochs=epochs)
        self.word2fasttext[0] = np.zeros(no_components)
        for word in self.word2id:
            self.word2fasttext[self.word2id[word]] = model.wv[word]
        return self.word2fasttext

    def word2MittensEmbedding(self, window_size=10, no_components=128, epochs=10, workers=os.cpu_count(), learning_rate=0.05):
        self.word2mittens = np.empty(shape=(self.no_words, no_components))
        vocab = list(self.word2id.keys())
        cooc_matrix = np.zeros((len(vocab), len(vocab)))
        for sentence in self.sentences:
            for i, word in enumerate(sentence):
                word_idx = self.word2id[word] - 1
                for j in range(max(0, i - window_size), min(len(sentence), i + window_size + 1)):
                    if i != j:
                        cooc_idx = self.word2id[sentence[j]] - 1
                        cooc_matrix[word_idx, cooc_idx] += 1

        # Convert to sparse matrix
        cooc_matrix_sparse = csr_matrix(cooc_matrix)
        model = Mittens(n=no_components, max_iter=epochs, mittens=learning_rate)
        try:
            embeddings = model.fit(cooc_matrix_sparse, vocab=vocab)
            self.word2mittens[0] = np.zeros(no_components)
            for word, idx in self.word2id.items():
                self.word2mittens[idx] = embeddings[idx - 1]
        except Exception as e:
            print(f"Error in Mittens.fit: {e}")
            embeddings = np.zeros((len(vocab), no_components))  # Fallback
            self.word2mittens[0] = np.zeros(no_components)
            for i, word in enumerate(vocab, 1):
                self.word2mittens[i] = embeddings[i - 1]
        return self.word2mittens

if __name__ == '__main__':
    corpus = [
        [['Hello', 'this', 'tutorial', 'on', 'how', 'convert', 'word', 'integer', 'format'],
         ['this', 'beautiful', 'day'],
         ['Jack', 'going', 'office']],
        [['Hello', 'this', 'tutorial', 'on', 'how', 'convert', 'word', 'integer', 'format'],
         ['this', 'beautiful', 'day'],
         ['Jack', 'going', 'office']],
        [['Hello', 'this', 'tutorial', 'on', 'how', 'convert', 'word', 'integer', 'format'],
         ['this', 'beautiful', 'day'],
         ['Jack', 'going', 'office']],
    ]

    we = WordEmbeddings(corpus)
    docs = we.prepareDocuments()
    print(np.array(docs, dtype=object).shape)
    print(docs)

    w2v = we.word2vecEmbedding()
    print("Word2Vec:", w2v.shape)
    print(w2v)

    w2f = we.word2FastTextEmbeddings()
    print("FastText:", w2f.shape)
    print(w2f)

    w2g = we.word2GloVeEmbedding()
    print("GloVe:", w2g.shape)
    print(w2g)

    w2m = we.word2MittensEmbedding()
    print("Mittens:", w2m.shape)
    print(w2m)

    print("\n\nComparison for word ID 1:")
    print("Word2Vec:", w2v[1])
    print("FastText:", w2f[1])
    print("GloVe:", w2g[1])
    print("Mittens:", w2m[1])

NumPy version: 1.26.4
Gensim version: 4.3.3
Mittens version: 0.2
(3, 15)
[[1, 2, 3, 4, 5, 6, 7, 8, 9, 2, 10, 11, 12, 13, 14], [1, 2, 3, 4, 5, 6, 7, 8, 9, 2, 10, 11, 12, 13, 14], [1, 2, 3, 4, 5, 6, 7, 8, 9, 2, 10, 11, 12, 13, 14]]
Word2Vec: (15, 128)
[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [-6.21626532e-05  3.19088437e-03 -7.12745590e-03 ...  9.92520479e-04
   7.14840600e-03  2.83964048e-03]
 [-3.82514030e-04  1.51936125e-04  3.97204095e-03 ...  2.31946167e-03
  -3.84517061e-03  3.48054501e-03]
 ...
 [-1.44330040e-03 -3.37802432e-03 -5.04739862e-03 ...  4.78180777e-03
   3.40414606e-03  2.03621481e-03]
 [ 5.68562094e-03 -4.50939965e-03  6.46589976e-03 ...  4.88198549e-03
  -7.37946481e-03  7.49228429e-03]
 [-1.35904271e-03  5.24326880e-03  7.78503902e-03 ... -6.09789602e-03
  -7.12639559e-03 -4.63685114e-03]]


Iteration 100: error 0.0001

FastText: (15, 128)
[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [-1.05216913e-03 -7.23858946e-04 -2.48263852e-04 ...  4.79545008e-04
   1.65466184e-03 -2.04305412e-04]
 [-1.36351539e-03  1.12285069e-03 -1.62398699e-03 ...  1.27794396e-03
  -4.78518370e-04  1.83548301e-03]
 ...
 [-2.22318270e-03  2.95931986e-05 -4.96662979e-04 ...  1.28600537e-03
   9.15645447e-04 -2.91286968e-04]
 [-3.23804357e-04  9.54689051e-04 -1.18813978e-03 ...  6.51234936e-04
   2.29042576e-04  1.05157425e-03]
 [-4.99513757e-04  2.49632495e-03  2.18920293e-03 ...  4.68000828e-04
  -1.34677067e-03 -1.70546729e-04]]
GloVe: (15, 128)
[[ 0.          0.          0.         ...  0.          0.
   0.        ]
 [-0.24973279 -0.47211752 -0.3471582  ... -0.01941465  0.43606771
  -0.0892403 ]
 [-0.2158313  -0.12271662 -0.21352648 ... -0.07537541  0.31608701
   0.02839591]
 ...
 [ 0.16518111  0.128661   -0.0561627  ... -0.03275083  0.00815867
  -0.02345717]
 [-0.0

### slimmed-down version without Mittens



In [8]:
# Verify versions after restart
import numpy as np
print("NumPy version:", np.__version__)

from gensim.models import Word2Vec, FastText
import gensim
print("Gensim version:", gensim.__version__)

from mittens import GloVe
import mittens
print("Mittens version (GloVe only):", mittens.__version__)

class WordEmbeddings:

    def __init__(self, corpus, normalize_tfidf=False):
        self.corpus = corpus
        self.normalize_tfidf = normalize_tfidf
        self.documents = []
        self.sentences = []
        self.word2id = {}
        self.no_words = 0
        self.max_size = 0
        self.no_docs = len(self.corpus)

    def prepareDocuments(self):
        word_id = 1
        for document in self.corpus:
            doc = []
            for sentence in document:
                self.sentences.append(sentence)
                for word in sentence:
                    if self.word2id.get(word) is None:
                        self.word2id[word] = word_id
                        word_id += 1
                    doc.append(self.word2id[word])
            if self.max_size < len(doc):
                self.max_size = len(doc)
            self.documents.append(doc)

        self.no_words = len(self.word2id) + 1
        return self.documents

    def word2vecEmbedding(self, window_size=10, no_components=128, epochs=10, workers=os.cpu_count(), sg=0, learning_rate=0.05):
        self.word2vec = np.empty(shape=(self.no_words, no_components))
        model = Word2Vec(self.sentences, vector_size=no_components, window=window_size, min_count=1,
                         workers=workers, sg=sg, alpha=learning_rate, epochs=epochs)
        self.word2vec[0] = np.zeros(no_components)
        for word in self.word2id:
            self.word2vec[self.word2id[word]] = model.wv[word]
        return self.word2vec

    def word2GloVeEmbedding(self, window_size=10, no_components=128, epochs=10, workers=os.cpu_count(), learning_rate=0.05):
        self.word2glove = np.empty(shape=(self.no_words, no_components))
        model = GloVe(n=no_components, learning_rate=learning_rate)

        vocab = list(self.word2id.keys())
        cooc_matrix = np.zeros((len(vocab), len(vocab)))
        for sentence in self.sentences:
            for i, word in enumerate(sentence):
                word_idx = self.word2id[word] - 1
                for j in range(max(0, i - window_size), min(len(sentence), i + window_size + 1)):
                    if i != j:
                        cooc_idx = self.word2id[sentence[j]] - 1
                        cooc_matrix[word_idx, cooc_idx] += 1

        embeddings = model.fit(cooc_matrix)
        self.word2glove[0] = np.zeros(no_components)
        for word, idx in self.word2id.items():
            self.word2glove[idx] = embeddings[idx - 1]
        return self.word2glove

    def word2FastTextEmbeddings(self, window_size=10, no_components=128, epochs=10, workers=os.cpu_count(), sg=0, learning_rate=0.05):
        self.word2fasttext = np.empty(shape=(self.no_words, no_components))
        model = FastText(self.sentences, vector_size=no_components, window=window_size, min_count=1,
                         workers=workers, sg=sg, alpha=learning_rate, epochs=epochs)
        self.word2fasttext[0] = np.zeros(no_components)
        for word in self.word2id:
            self.word2fasttext[self.word2id[word]] = model.wv[word]
        return self.word2fasttext

if __name__ == '__main__':
    corpus = [
        [['Hello', 'this', 'tutorial', 'on', 'how', 'convert', 'word', 'integer', 'format'],
         ['this', 'beautiful', 'day'],
         ['Jack', 'going', 'office']],
        [['Hello', 'this', 'tutorial', 'on', 'how', 'convert', 'word', 'integer', 'format'],
         ['this', 'beautiful', 'day'],
         ['Jack', 'going', 'office']],
        [['Hello', 'this', 'tutorial', 'on', 'how', 'convert', 'word', 'integer', 'format'],
         ['this', 'beautiful', 'day'],
         ['Jack', 'going', 'office']],
    ]

    we = WordEmbeddings(corpus)
    docs = we.prepareDocuments()
    print(np.array(docs, dtype=object).shape)
    print(docs)

    w2v = we.word2vecEmbedding()
    print("Word2Vec:", w2v.shape)
    print(w2v)

    w2f = we.word2FastTextEmbeddings()
    print("FastText:", w2f.shape)
    print(w2f)

    w2g = we.word2GloVeEmbedding()
    print("GloVe:", w2g.shape)
    print(w2g)

    print("\n\nComparison for word ID 1:")
    print("Word2Vec:", w2v[1])
    print("FastText:", w2f[1])
    print("GloVe:", w2g[1])

NumPy version: 1.26.4
Gensim version: 4.3.3
Mittens version (GloVe only): 0.2
(3, 15)
[[1, 2, 3, 4, 5, 6, 7, 8, 9, 2, 10, 11, 12, 13, 14], [1, 2, 3, 4, 5, 6, 7, 8, 9, 2, 10, 11, 12, 13, 14], [1, 2, 3, 4, 5, 6, 7, 8, 9, 2, 10, 11, 12, 13, 14]]
Word2Vec: (15, 128)
[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [-6.21626532e-05  3.19088437e-03 -7.12745590e-03 ...  9.92520479e-04
   7.14840600e-03  2.83964048e-03]
 [-3.82514030e-04  1.51936125e-04  3.97204095e-03 ...  2.31946167e-03
  -3.84517061e-03  3.48054501e-03]
 ...
 [-1.44330040e-03 -3.37802432e-03 -5.04739862e-03 ...  4.78180777e-03
   3.40414606e-03  2.03621481e-03]
 [ 5.68562094e-03 -4.50939965e-03  6.46589976e-03 ...  4.88198549e-03
  -7.37946481e-03  7.49228429e-03]
 [-1.35904271e-03  5.24326880e-03  7.78503902e-03 ... -6.09789602e-03
  -7.12639559e-03 -4.63685114e-03]]


Iteration 100: error 0.0000

FastText: (15, 128)
[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [-1.05216913e-03 -7.23858946e-04 -2.48263852e-04 ...  4.79545008e-04
   1.65466184e-03 -2.04305412e-04]
 [-1.36351539e-03  1.12285069e-03 -1.62398699e-03 ...  1.27794396e-03
  -4.78518370e-04  1.83548301e-03]
 ...
 [-2.22318270e-03  2.95931986e-05 -4.96662979e-04 ...  1.28600537e-03
   9.15645447e-04 -2.91286968e-04]
 [-3.23804357e-04  9.54689051e-04 -1.18813978e-03 ...  6.51234936e-04
   2.29042576e-04  1.05157425e-03]
 [-4.99513757e-04  2.49632495e-03  2.18920293e-03 ...  4.68000828e-04
  -1.34677067e-03 -1.70546729e-04]]
GloVe: (15, 128)
[[ 0.          0.          0.         ...  0.          0.
   0.        ]
 [-0.14491981 -0.09447325  0.12144629 ... -0.2624261  -0.18998113
  -0.09980709]
 [ 0.32938427 -0.05994402  0.05077884 ...  0.01185149  0.20612515
   0.28760553]
 ...
 [-0.01241104 -0.08506157  0.29257517 ... -0.17227083 -0.19028408
  -0.19335479]
 [ 0.3

## tokenization

In [10]:
# Install required packages and download NLTK data
!pip install numpy==1.26.4 gensim==4.3.3 mittens==0.2 spacy==3.7.2 stop-words==2018.7.23 -q
!python -m spacy download en_core_web_sm -q
import nltk
nltk.download('stopwords', quiet=True)

import re
import spacy
from stop_words import get_stop_words
from nltk.corpus import stopwords
import numpy as np
from gensim.models import Word2Vec, FastText
from mittens import GloVe
import os

# Special characters dictionary
specialchar_dic = {
    "’": "'", "„": "\"", "“": "\"", "”": "\"", "«": "<<", "»": ">>",
    "…": "...", "—": "--", "¡": "!", "¿": "?", "©": " ", "–": " "
}

# Stop words function (cached globally)
def stopWordsEN():
    sw_stop_words = get_stop_words('en')
    sw_nltk = stopwords.words('english')
    sw_spacy = list(spacy.lang.en.stop_words.STOP_WORDS)
    sw_mallet = ['a', 'able', 'about', 'above', 'according', 'accordingly', 'across', 'actually', 'after', 'afterwards', 'again', 'against', 'all', 'allow', 'allows', 'almost', 'alone', 'along', 'already', 'also', 'although', 'always', 'am', 'among', 'amongst', 'an', 'and', 'another', 'any', 'anybody', 'anyhow', 'anyone', 'anything', 'anyway', 'anyways', 'anywhere', 'apart', 'appear', 'appreciate', 'appropriate', 'are', 'around', 'as', 'aside', 'ask', 'asking', 'associated', 'at', 'available', 'away', 'awfully', 'b', 'be', 'became', 'because', 'become', 'becomes', 'becoming', 'been', 'before', 'beforehand', 'behind', 'being', 'believe', 'below', 'beside', 'besides', 'best', 'better', 'between', 'beyond', 'both', 'brief', 'but', 'by', 'c', 'came', 'can', 'cannot', 'cant', 'cause', 'causes', 'certain', 'certainly', 'changes', 'clearly', 'co', 'com', 'come', 'comes', 'concerning', 'consequently', 'consider', 'considering', 'contain', 'containing', 'contains', 'corresponding', 'could', 'course', 'currently', 'd', 'definitely', 'described', 'despite', 'did', 'different', 'do', 'does', 'doing', 'done', 'down', 'downwards', 'during', 'e', 'each', 'edu', 'eg', 'eight', 'either', 'else', 'elsewhere', 'enough', 'entirely', 'especially', 'et', 'etc', 'even', 'ever', 'every', 'everybody', 'everyone', 'everything', 'everywhere', 'ex', 'exactly', 'example', 'except', 'f', 'far', 'few', 'fifth', 'first', 'five', 'followed', 'following', 'follows', 'for', 'former', 'formerly', 'forth', 'four', 'from', 'further', 'furthermore', 'g', 'get', 'gets', 'getting', 'given', 'gives', 'go', 'goes', 'going', 'gone', 'got', 'gotten', 'greetings', 'h', 'had', 'happens', 'hardly', 'has', 'have', 'having', 'he', 'hello', 'help', 'hence', 'her', 'here', 'hereafter', 'hereby', 'herein', 'hereupon', 'hers', 'herself', 'hi', 'him', 'himself', 'his', 'hither', 'hopefully', 'how', 'howbeit', 'however', 'i', 'ie', 'if', 'ignored', 'immediate', 'in', 'inasmuch', 'inc', 'indeed', 'indicate', 'indicated', 'indicates', 'inner', 'insofar', 'instead', 'into', 'inward', 'is', 'it', 'its', 'itself', 'j', 'just', 'k', 'keep', 'keeps', 'kept', 'know', 'knows', 'known', 'l', 'last', 'lately', 'later', 'latter', 'latterly', 'least', 'less', 'lest', 'let', 'like', 'liked', 'likely', 'little', 'look', 'looking', 'looks', 'ltd', 'm', 'mainly', 'many', 'may', 'maybe', 'me', 'mean', 'meanwhile', 'merely', 'might', 'more', 'moreover', 'most', 'mostly', 'much', 'must', 'my', 'myself', 'n', 'name', 'namely', 'nd', 'near', 'nearly', 'necessary', 'need', 'needs', 'neither', 'never', 'nevertheless', 'new', 'next', 'nine', 'no', 'nobody', 'non', 'none', 'noone', 'nor', 'normally', 'not', 'nothing', 'novel', 'now', 'nowhere', 'o', 'obviously', 'of', 'off', 'often', 'oh', 'ok', 'okay', 'old', 'on', 'once', 'one', 'ones', 'only', 'onto', 'or', 'other', 'others', 'otherwise', 'ought', 'our', 'ours', 'ourselves', 'out', 'outside', 'over', 'overall', 'own', 'p', 'particular', 'particularly', 'per', 'perhaps', 'placed', 'please', 'plus', 'possible', 'presumably', 'probably', 'provides', 'q', 'que', 'quite', 'qv', 'r', 'rather', 'rd', 're', 'really', 'reasonably', 'regarding', 'regardless', 'regards', 'relatively', 'respectively', 'right', 's', 'said', 'same', 'saw', 'say', 'saying', 'says', 'second', 'secondly', 'see', 'seeing', 'seem', 'seemed', 'seeming', 'seems', 'seen', 'self', 'selves', 'sensible', 'sent', 'serious', 'seriously', 'seven', 'several', 'shall', 'she', 'should', 'since', 'six', 'so', 'some', 'somebody', 'somehow', 'someone', 'something', 'sometime', 'sometimes', 'somewhat', 'somewhere', 'soon', 'sorry', 'specified', 'specify', 'specifying', 'still', 'sub', 'such', 'sup', 'sure', 't', 'take', 'taken', 'tell', 'tends', 'th', 'than', 'thank', 'thanks', 'thanx', 'that', 'thats', 'the', 'their', 'theirs', 'them', 'themselves', 'then', 'thence', 'there', 'thereafter', 'thereby', 'therefore', 'therein', 'theres', 'thereupon', 'these', 'they', 'think', 'third', 'this', 'thorough', 'thoroughly', 'those', 'though', 'three', 'through', 'throughout', 'thru', 'thus', 'to', 'together', 'too', 'took', 'toward', 'towards', 'tried', 'tries', 'truly', 'try', 'trying', 'twice', 'two', 'u', 'un', 'under', 'unfortunately', 'unless', 'unlikely', 'until', 'unto', 'up', 'upon', 'us', 'use', 'used', 'useful', 'uses', 'using', 'usually', 'uucp', 'v', 'value', 'various', 'very', 'via', 'viz', 'vs', 'w', 'want', 'wants', 'was', 'way', 'we', 'welcome', 'well', 'went', 'were', 'what', 'whatever', 'when', 'whence', 'whenever', 'where', 'whereafter', 'whereas', 'whereby', 'wherein', 'whereupon', 'wherever', 'whether', 'which', 'while', 'whither', 'who', 'whoever', 'whole', 'whom', 'whose', 'why', 'will', 'willing', 'wish', 'with', 'within', 'without', 'wonder', 'would', 'x', 'y', 'yes', 'yet', 'you', 'your', 'yours', 'yourself', 'yourselves', 'z', 'zero']
    return list(set(sw_stop_words + sw_nltk + sw_mallet + sw_spacy))

# Precompile regex and load Spacy model
punctuation = '!"#$%&\'()*+,./:;<=>?@[\\]^_`{|}~-'
specialchar_re = re.compile('(%s)' % '|'.join(specialchar_dic.keys()))
nlp = spacy.load("en_core_web_sm")
cachedStopWords_en = stopWordsEN()

class Tokenization:
    def applyFE(self, text):
        """Combine negation with words to reduce bias."""
        final_text = text.replace('cannot', 'can not').replace('can\'t', 'can not')
        final_text = final_text.replace('won\'t', 'will not').replace('n\'t', ' not').replace(' not ', ' not')
        return final_text

    def removeStopWords(self, text):
        return ' '.join([word for word in text.split() if word not in cachedStopWords_en])

    def removePunctuation(self, text, punctuation=punctuation):
        for c in punctuation:
            text = text.replace(c, ' ')
        return text

    def replaceUTF8Char(self, text, specialchars=specialchar_dic):
        def replace(match):
            return specialchars[match.group(0)]
        return specialchar_re.sub(replace, text)

    def createCorpus(self, text, remove_punctuation=True, remove_stopwords=True, apply_FE=True):
        corpus = []
        try:
            text = self.replaceUTF8Char(text).replace("\n", " ")
            doc = nlp(text)
            processed_text = ' '.join([t.lemma_ if t.lemma_ != '-PRON-' else t.text if not t.ent_type_ else t.text for t in doc])
            processed_text = processed_text.replace("\s\s+", ' ')

            doc = nlp(processed_text.lower())
            rawText = not (remove_punctuation or remove_stopwords or apply_FE)

            for sentence in doc.sents:
                sent = str(sentence.text)
                if len(sent) == 0:
                    continue
                if not rawText:
                    if apply_FE:
                        sent = self.applyFE(text=sent)
                    if remove_punctuation:
                        sent = self.removePunctuation(text=sent)
                    if remove_stopwords:
                        sent = self.removeStopWords(text=sent)
                sent = sent.lower().split()
                if sent:
                    corpus.append(sent)
        except Exception as exp:
            print('exception=', str(exp))
            print('text=', text)
        return corpus

    def __del__(self):
        print("Destructor Tokenization")

class WordEmbeddings:
    def __init__(self, corpus, normalize_tfidf=False):
        self.corpus = corpus
        self.normalize_tfidf = normalize_tfidf
        self.documents = []
        self.sentences = []
        self.word2id = {}
        self.no_words = 0
        self.max_size = 0
        self.no_docs = len(self.corpus)

    def prepareDocuments(self):
        word_id = 1
        for document in self.corpus:
            doc = []
            for sentence in document:
                self.sentences.append(sentence)
                for word in sentence:
                    if self.word2id.get(word) is None:
                        self.word2id[word] = word_id
                        word_id += 1
                    doc.append(self.word2id[word])
            if self.max_size < len(doc):
                self.max_size = len(doc)
            self.documents.append(doc)

        self.no_words = len(self.word2id) + 1
        return self.documents

    def word2vecEmbedding(self, window_size=10, no_components=128, epochs=10, workers=os.cpu_count(), sg=0, learning_rate=0.05):
        self.word2vec = np.empty(shape=(self.no_words, no_components))
        model = Word2Vec(self.sentences, vector_size=no_components, window=window_size, min_count=1,
                         workers=workers, sg=sg, alpha=learning_rate, epochs=epochs)
        self.word2vec[0] = np.zeros(no_components)
        for word in self.word2id:
            self.word2vec[self.word2id[word]] = model.wv[word]
        return self.word2vec

    def word2GloVeEmbedding(self, window_size=10, no_components=128, epochs=10, workers=os.cpu_count(), learning_rate=0.05):
        self.word2glove = np.empty(shape=(self.no_words, no_components))
        model = GloVe(n=no_components, learning_rate=learning_rate)

        vocab = list(self.word2id.keys())
        cooc_matrix = np.zeros((len(vocab), len(vocab)))
        for sentence in self.sentences:
            for i, word in enumerate(sentence):
                word_idx = self.word2id[word] - 1
                for j in range(max(0, i - window_size), min(len(sentence), i + window_size + 1)):
                    if i != j:
                        cooc_idx = self.word2id[sentence[j]] - 1
                        cooc_matrix[word_idx, cooc_idx] += 1

        embeddings = model.fit(cooc_matrix)
        self.word2glove[0] = np.zeros(no_components)
        for word, idx in self.word2id.items():
            self.word2glove[idx] = embeddings[idx - 1]
        return self.word2glove

    def word2FastTextEmbeddings(self, window_size=10, no_components=128, epochs=10, workers=os.cpu_count(), sg=0, learning_rate=0.05):
        self.word2fasttext = np.empty(shape=(self.no_words, no_components))
        model = FastText(self.sentences, vector_size=no_components, window=window_size, min_count=1,
                         workers=workers, sg=sg, alpha=learning_rate, epochs=epochs)
        self.word2fasttext[0] = np.zeros(no_components)
        for word in self.word2id:
            self.word2fasttext[self.word2id[word]] = model.wv[word]
        return self.word2fasttext

if __name__ == '__main__':
    # Tokenization example
    tkn = Tokenization()
    text1 = "Apple data-intensive is looking at buying U.K. startup for $1 billion. This is great! The new D.P. model is funcitonal and ready"
    corpus1 = tkn.createCorpus(text1)
    print("Corpus 1:", corpus1)

    # Larger text example
    text2 = """The lion may be known as the king of the jungle, but lions do not live in jungles. They’re the rulers of the African savannahs that are covered in brown grasses and speckled with sparse trees. Lions’ coloring helps them blend in perfectly with the tall grass so they can ambush their prey as best as possible. And lions are ferocious. Although they’re one of the most powerful predators on land, lions are in danger. Hunters and poachers target lions to prove to the world their machismo.\n\nAnd while hunters seek to wipe lions off the face of the earth to bolster their egos, the Kevin Richardson Wildlife Sanctuary hopes to stop them and protect the big African cat at all cost.\n\nRichardson has earned the nickname the “Lion Whisperer” for a reason. He aims to educate the world about lions. And for those lucky enough to volunteer alongside Richardson, he encourages them to learn more about lions and help protect the wild species.\n\n“To raise awareness, Kevin has now set up his YouTube Channel ‘LionWhispererTV’. The channel is all about raising awareness about not only the declining numbers of lions but also how this rapid decrease is happening. By watching these videos, you are directly contributing to our scheme of land acquisition,” he writes in his bio.\n\nAs part of the volunteer program, Richardson hosts a “volunteer enrichment and lion enrichment” walk. As the name suggests, Richardson takes his group of volunteers out into the savannah of South Africa to hang out with two lions. There, the volunteers meet a male lion, Bobcat, and a female lioness, Gabby. Both lions look ferocious, but are truly “affectionate,” at least that’s what Richardson says. And remember, he’s the lion whisperer, so he’s got an advantage with these deadly big cats.\n\nAs Richardson showers the pair of lions with love, the volunteers stay locked in the truck, unwilling to put their lives in danger. And while they are in the vehicle, the lions are just feet from them – and if something goes wrong, they could wind up injured anyway.\n\nRichardson shared the video on his “The Lion Whisperer” YouTube channel. With more than one million hits, this video has proven to be one of his most famous.\n\nThe video describes the moment caught on tape as follows:\n\n“It’s an enrichment walk for both the volunteers and the lions as Kevin shows off his lovely lions as well as giving some amazing lion facts to the volunteers.”\n\nViewers like you are overwhelmed with the magnificent footage. The following are a few comments shared on the video.\n\n“I hope to someday volunteer there with Kevin. I believe in the work and his perspective about conservation. This video makes me want to all the more! Bobcat and Gabby are lovely lions.” “Every time I watch a one of your videos I somehow end up smiling from ear to ear!” “That was so beautiful, wish I could rub my head against a lion.”\n\nTake a moment to watch this video. Would you ever want to volunteer with Kevin Richardson and his lions?"""
    corpus2 = tkn.createCorpus(text2, remove_stopwords=False)
    print("Corpus 2:", corpus2)

    # Generate embeddings from Corpus 2
    we = WordEmbeddings(corpus2)
    docs = we.prepareDocuments()
    print("\nDocuments shape:", np.array(docs, dtype=object).shape)
    print("Documents:", docs)

    w2v = we.word2vecEmbedding()
    print("Word2Vec shape:", w2v.shape)
    print("Word2Vec embeddings:", w2v[:5])  # Print first 5 for brevity

    w2f = we.word2FastTextEmbeddings()
    print("FastText shape:", w2f.shape)
    print("FastText embeddings:", w2f[:5])

    w2g = we.word2GloVeEmbedding()
    print("GloVe shape:", w2g.shape)
    print("GloVe embeddings:", w2g[:5])

    print("\nComparison for word 'lion' (ID varies):")
    lion_id = we.word2id.get('lion', -1)
    if lion_id != -1:
        print("Word2Vec:", w2v[lion_id])
        print("FastText:", w2f[lion_id])
        print("GloVe:", w2g[lion_id])
    else:
        print("'lion' not found in vocabulary")

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 920.2/920.2 kB 54.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 108.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 91.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


/usr/local/lib/python3.11/dist-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_core_web_sm' (3.7.1) was trained with spaCy v3.7.2 and may not be 100% compatible with the current version (3.8.4). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


Corpus 1: [['apple', 'data', 'intensive', 'buy', 'startup', '1', 'billion'], ['great'], ['model', 'funcitonal', 'ready']]


Corpus 2: [['the', 'lion', 'may', 'be', 'know', 'as', 'the', 'king', 'of', 'the', 'jungle', 'but', 'lion', 'do', 'notlive', 'in', 'jungle'], ['they', 'be', 'the', 'ruler', 'of', 'the', 'african', 'savannah', 'that', 'be', 'cover', 'in', 'brown', 'grass', 'and', 'speckle', 'with', 'sparse', 'tree'], ['lion', 'coloring', 'help', 'they', 'blend', 'in', 'perfectly', 'with', 'the', 'tall', 'grass', 'so', 'they', 'can', 'ambush', 'their', 'prey', 'as', 'well', 'as', 'possible'], ['and', 'lion', 'be', 'ferocious'], ['although', 'they', 'be', 'one', 'of', 'the', 'most', 'powerful', 'predator', 'on', 'land', 'lion', 'be', 'in', 'danger'], ['hunter', 'and', 'poacher', 'target', 'lion', 'to', 'prove', 'to', 'the', 'world', 'their', 'machismo'], ['and', 'while', 'hunter', 'seek', 'to', 'wipe', 'lion', 'off', 'the', 'face', 'of', 'the', 'earth', 'to', 'bolster', 'their', 'ego', 'the', 'kevin', 'richardson', 'wildlife', 'sanctuary', 'hope', 'to', 'stop', 'they', 'and', 'protect', 'the', 'big', 'afri

Iteration 100: error 0.1536

FastText shape: (26, 128)
FastText embeddings: [[ 0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.

In [12]:
# Install dependencies
!pip install numpy==1.26.4 gensim==4.3.3 mittens==0.2 spacy==3.7.2 stop-words==2018.7.23 pandas scipy -q
!python -m spacy download en_core_web_sm -q
import nltk
nltk.download('stopwords', quiet=True)

import pandas as pd
from scipy import io as sio
import numpy as np
from multiprocessing import cpu_count
from concurrent.futures import ProcessPoolExecutor
from sklearn.preprocessing import StandardScaler
import sys

# Tokenization class (from earlier)
specialchar_dic = {
 "’": "'", "„": "\"", "“": "\"", "”": "\"", "«": "<<", "»": ">>",
 "…": "...", "—": "--", "¡": "!", "¿": "?", "©": " ", "–": " "
}
punctuation = '!"#$%&\'()*+,./:;<=>?@[\\]^_`{|}~-'
specialchar_re = re.compile('(%s)' % '|'.join(specialchar_dic.keys()))
nlp = spacy.load("en_core_web_sm")
cachedStopWords_en = stopWordsEN() # Assume stopWordsEN() is defined as before

class Tokenization:
 def applyFE(self, text):
 final_text = text.replace('cannot', 'can not').replace('can\'t', 'can not')
 final_text = final_text.replace('won\'t', 'will not').replace('n\'t', ' not').replace(' not ', ' not')
 return final_text

 def removeStopWords(self, text):
 return ' '.join([word for word in text.split() if word not in cachedStopWords_en])

 def removePunctuation(self, text, punctuation=punctuation):
 for c in punctuation:
 text = text.replace(c, ' ')
 return text

 def replaceUTF8Char(self, text, specialchars=specialchar_dic):
 def replace(match):
 return specialchars[match.group(0)]
 return specialchar_re.sub(replace, text)

 def createCorpus(self, text, remove_punctuation=True, remove_stopwords=True, apply_FE=True):
 corpus = []
 try:
 text = self.replaceUTF8Char(text).replace("\n", " ")
 doc = nlp(text)
 processed_text = ' '.join([t.lemma_ if t.lemma_ != '-PRON-' else t.text if not t.ent_type_ else t.text for t in doc])
 processed_text = processed_text.replace("\s\s+", ' ')
 doc = nlp(processed_text.lower())
 rawText = not (remove_punctuation or remove_stopwords or apply_FE)
 for sentence in doc.sents:
 sent = str(sentence.text)
 if len(sent) == 0:
 continue
 if not rawText:
 if apply_FE:
 sent = self.applyFE(text=sent)
 if remove_punctuation:
 sent = self.removePunctuation(text=sent)
 if remove_stopwords:
 sent = self.removeStopWords(text=sent)
 sent = sent.lower().split()
 if sent:
 corpus.append(sent)
 except Exception as exp:
 print('exception=', str(exp))
 print('text=', text)
 return corpus

# WordEmbeddings class (without Mittens)
class WordEmbeddings:
 def __init__(self, corpus, normalize_tfidf=False):
 self.corpus = corpus
 self.normalize_tfidf = normalize_tfidf
 self.documents = []
 self.sentences = []
 self.word2id = {}
 self.no_words = 0
 self.max_size = 0
 self.no_docs = len(self.corpus)

 def prepareDocuments(self):
 word_id = 1
 for document in self.corpus:
 doc = []
 for sentence in document:
 self.sentences.append(sentence)
 for word in sentence:
 if self.word2id.get(word) is None:
 self.word2id[word] = word_id
 word_id += 1
 doc.append(self.word2id[word])
 if self.max_size < len(doc):
 self.max_size = len(doc)
 self.documents.append(doc)
 self.no_words = len(self.word2id) + 1
 return self.documents

 def word2vecEmbedding(self, window_size=10, no_components=128, epochs=10, workers=os.cpu_count(), sg=0, learning_rate=0.05):
 self.word2vec = np.empty(shape=(self.no_words, no_components))
 model = Word2Vec(self.sentences, vector_size=no_components, window=window_size, min_count=1,
 workers=workers, sg=sg, alpha=learning_rate, epochs=epochs)
 self.word2vec[0] = np.zeros(no_components)
 for word in self.word2id:
 self.word2vec[self.word2id[word]] = model.wv[word]
 return self.word2vec

 def word2GloVeEmbedding(self, window_size=10, no_components=128, epochs=10, workers=os.cpu_count(), learning_rate=0.05):
 self.word2glove = np.empty(shape=(self.no_words, no_components))
 model = GloVe(n=no_components, learning_rate=learning_rate)
 vocab = list(self.word2id.keys())
 cooc_matrix = np.zeros((len(vocab), len(vocab)))
 for sentence in self.sentences:
 for i, word in enumerate(sentence):
 word_idx = self.word2id[word] - 1
 for j in range(max(0, i - window_size), min(len(sentence), i + window_size + 1)):
 if i != j:
 cooc_idx = self.word2id[sentence[j]] - 1
 cooc_matrix[word_idx, cooc_idx] += 1
 embeddings = model.fit(cooc_matrix)
 self.word2glove[0] = np.zeros(no_components)
 for word, idx in self.word2id.items():
 self.word2glove[idx] = embeddings[idx - 1]
 return self.word2glove

 def word2FastTextEmbeddings(self, window_size=10, no_components=128, epochs=10, workers=os.cpu_count(), sg=0, learning_rate=0.05):
 self.word2fasttext = np.empty(shape=(self.no_words, no_components))
 model = FastText(self.sentences, vector_size=no_components, window=window_size, min_count=1,
 workers=workers, sg=sg, alpha=learning_rate, epochs=epochs)
 self.word2fasttext[0] = np.zeros(no_components)
 for word in self.word2id:
 self.word2fasttext[self.word2id[word]] = model.wv[word]
 return self.word2fasttext

# Helper function for parallel processing
def processElement(elem):
 id_line = elem[0]
 text = elem[1]
 tkn = Tokenization()
 text = tkn.createCorpus(text, remove_stopwords=False)
 return id_line, text

if __name__ == "__main__":
 # For Colab, replace sys.argv[1] with a hardcoded file path or upload mechanism
 # Example: Upload file in Colab
 from google.colab import files
 uploaded = files.upload()
 fn = list(uploaded.keys())[0] # Get the uploaded filename
 df = pd.read_csv(fn, sep=',')

 print(df)
 labels = df['label'].unique()
 num_classes = len(labels)
 label2id = {'mostly true': 0, 'mixture of true and false': 1, 'no factual content': 1, 'mostly false': 1}
 for label in labels:
 df.loc[df['label'] == label, 'label'] = label2id[label]

 y = df['label'].astype(int).to_list()
 sio.savemat('labels.mat', {'y': y})

 X_network = df[['num_comments', 'num_shares', 'num_likes', 'num_loves', 'num_wows', 'num_hahas', 'num_sads', 'num_angrys']].to_numpy()
 scaler_std = StandardScaler()
 X_net_std = scaler_std.fit_transform(X_network)
 X_net_std = X_net_std.reshape((X_net_std.shape[0], 1, X_net_std.shape[1]))
 print("X_network:", X_network)
 print("X_net_std:", X_net_std)
 sio.savemat('network.mat', {'X_net_std': X_net_std})

 print("Start Tokenization")
 texts = df[['id', 'content']].to_numpy().tolist()
 corpus = [None] * len(texts)
 no_threads = cpu_count() - 1
 with ProcessPoolExecutor(max_workers=no_threads) as worker:
 for result in worker.map(processElement, texts):
 if result:
 corpus[result[0]] = result[1]

 print("Corpus sample:")
 for idx, doc in enumerate(corpus[:5]): # Show first 5 for brevity
 print(idx, doc)

 print("Start Document Tokenization")
 we = WordEmbeddings(corpus)
 documents = we.prepareDocuments()
 vocabulary_size = we.no_words
 max_size = we.max_size
 print("Vocabulary size:", vocabulary_size)
 print("Max Document size:", max_size)

 X_docs = []
 for document in documents:
 doc_size = len(document)
 X_docs.append(document + [0] * (max_size - doc_size))
 X_docs = np.array(X_docs)
 sio.savemat('corpus.mat', {'X': X_docs})

 print("Start W2V CBOW")
 w2v_cbow = we.word2vecEmbedding(sg=0)
 sio.savemat('w2v_cbow.mat', {'w2v_cbow': w2v_cbow})

 print("Start W2V SG")
 w2v_sg = we.word2vecEmbedding(sg=1)
 sio.savemat('w2v_sg.mat', {'w2v_sg': w2v_sg})

 print("Start FT CBOW")
 ft_cbow = we.word2FastTextEmbeddings(sg=0)
 sio.savemat('ft_cbow.mat', {'ft_cbow': ft_cbow})

 print("Start FT SG")
 ft_sg = we.word2FastTextEmbeddings(sg=1)
 sio.savemat('ft_sg.mat', {'ft_sg': ft_sg})

 print("Start GLOVE")
 glove = we.word2GloVeEmbedding()
 sio.savemat('glove.mat', {'glove': glove})

IndentationError: expected an indented block after function definition on line 26 (<ipython-input-12-37b5f3cf75e6>, line 27)